In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tradingWithPython as twp
import tradingWithPython.lib.cboe as cboe
import tradingWithPython.lib.yahooFinance as yf
import pandas as pd

In [9]:
spy = yf.getHistoricData('SPY')['adj_close']
vxx = yf.getHistoricData('VXX')['adj_close']
vxz = yf.getHistoricData('VXZ')['adj_close']
vix = cboe.getHistoricData('VIX')
vxv = cboe.getHistoricData('VXV')

Got 6049 days of data
Got 2018 days of data
Got 2004 days of data


In [17]:
df = pd.DataFrame({'SPY':spy, 'VXX':vxx,'VXZ':vxz, 'VIX':vix, 'VXV':vxv}).dropna()
#请教一下老师吧？

ValueError: could not broadcast input array from shape (2309) into shape (3296)

In [7]:
df = pd.DataFrame({'SPY':spy, 'VXX':vxx, 'VIX':vix, 'VXV':vxv}).dropna()

C:\Users\User\Anaconda2\envs\py35\lib\site-packages\pandas\tseries\index.py:1007: RuntimeWarning: Cannot compare type 'Timestamp' with type 'int', sort order is undefined for incomparable objects
  this = Index.union(this, other)


AttributeError: 'Index' object has no attribute 'freq'